# 隐藏层

我们已经成功地训练了一个线性回归的机器学习模型，用来根据天气预报（温度、湿度）预测冰激凌销量。

但是冰激凌的销量是直接和天气情况相关联吗？实际上，天气情况决定了人们出门的愿望，和购买冰激凌的欲望。而人们出门的愿望，和购买冰激凌的欲望才直接决定了冰激凌的销量。

比如天气凉爽的时候，人们愿意出门，却未必愿意购买冰激凌；而天气酷热的时候，人们不愿意出门，但是有购买冰激凌的强烈愿望；等到天气寒冷的时候，人们即不愿意出门，也不想购买冰激凌。

所以，理论上我们需要三个模型：第一个模型根据天气情况预测人们出门的愿望，第二个模型根据天气情况预测人们购买冰激凌的欲望；而第三个模型根据前两个模型的预测结果来预测冰激凌的销量。

训练这样的三个模型，我们需要收集四组数据：
* 天气预报
* 顾客出门比例
* 冰激凌购买比例
* 冰激凌销量

这里面的顾客出门比例，和冰激凌购买比例被称为中间数据。它们并不是我们直接需要的数据，也很难收集。

## 层

我们设想一下，是否可以把我们的模型分成两**层**（Layer）：

* 第一层：根据天气情况预测两个数据：人们出门的愿望，和购买冰激凌的欲望。
* 第二层：利用第一层的两个预测结果来预测冰激凌的销量。

这样的模型是否更加合理？是否可行？


In [82]:
import numpy as np

np.random.seed(99)

## 数据集

### 训练数据：特征、标签

In [83]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63]])

train_labels = np.array([[95],
                        [210],
                        [70],
                        [155]])

### 测试数据：特征、标签

In [84]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165]])

## 模型

两层的人工神经元网络模型，其数据流将变成传播链路：

* 前向传播链路

```{figure} layer-forward.png
:align: center
:width: 480px
```

* 梯度计算链路

```{figure} layer-gradient.png
:align: center
:width: 680px
```

* 反向传播链路

```{figure} layer-backward.png
:align: center
:width: 260px
```

两层模型训练的过程是多轮迭代，通过两层梯度，逐步调整两层参数，收敛到最优解。

根据微积分的导数链式法则，两层模型的反向传播链是可行的。而且这个过程不需要中间值的参与，因此我们可以忽略中间值的实际含义。

### 参数：隐藏层权重、偏差

实际上，我们并没有办法要求模型在第一层必须预测人们出门的愿望，和购买冰激凌的欲望。除非我们可以提供有关人们出门的愿望，和购买冰激凌的欲望的实际数据给模型学习。

我们只是要求模型在第一层先推理出若干中间值。这些中间值没有明确的实际含义，但是将成为第二层的更大数量、更高质量的训练数据。

因此第一层通常称为**隐藏层**（Hidden Layer）。我们设置隐藏层推理出四个中间值。

我们使用NumPy的随机数生成器，初始化四组随机数权重和偏差。这样的设置将引导模型沿着不同方向推理中间值。

In [85]:
hidden_weight = np.random.rand(4, 2) / 2
hidden_bias = np.random.rand(4)

### 参数：输出层权重、偏差

第二层通常称为**输出层**（Output Layer）。我们同样使用NumPy的随机数生成器来初始化权重和偏差。

输出层将有四个输入数据，并推理出一个输出数据。

In [86]:
output_weight = np.random.rand(1, 4) / 4
output_bias = np.random.rand(1)

### 推理函数

In [87]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [88]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

In [89]:
def gradient(p, y):
    return - 2 * (y - p) / len(y)

### 反向函数

隐藏层将产生四个中间值。我们需要调整一下反向函数关于偏差的计算：不再简单计算总和，而是按照批次的维度求和。这样可以保证每个中间值拥有不同的偏差。

In [90]:
def backward(x, d, w, b, lr):
    w = w - d.T @ x * lr
    b = b - np.sum(d, axis=0) * lr
    return w, b

### 梯度反向函数

梯度反向函数用于从输出层梯度推导出隐藏层梯度。其本质就是输出层推理函数的偏导数。

In [91]:
def gradient_backward(d, w):
    return d @ w

## 训练

### 超参数：学习率

In [92]:
LEARNING_RATE = 0.00001

### 超参数：批大小

In [93]:
BATCH_SIZE = 2

### 超参数：轮数

In [94]:
EPOCHS = 1000

### 迭代

单层网络模型的反向传播链包括损失函数的偏导，和推理函数的偏导两部分。两层网络模型的反向传播链包括三部分：损失函数的偏导，输出层推理函数的偏导，和隐藏层推理函数的偏导。

In [95]:
for epoch in range(EPOCHS):
    for i in range(0, len(train_features), BATCH_SIZE):
        # 读入一个批次需要的多个训练数据：特征、标签
        features = train_features[i: i + BATCH_SIZE]
        labels = train_labels[i: i + BATCH_SIZE]

        # 推理中间数据
        hidden = forward(features, hidden_weight, hidden_bias)
        # 推理输出数据
        predictions = forward(hidden, output_weight, output_bias)
        # 计算输出层梯度
        output_delta = gradient(predictions, labels)
        # 计算隐藏层梯度
        hidden_delta = gradient_backward(output_delta, output_weight)
        # 用输出层梯度更新输出层参数
        output_weight, output_bias = backward(hidden, output_delta, output_weight, output_bias, LEARNING_RATE)
        # 用隐藏层梯度更新隐藏层参数
        hidden_weight, hidden_bias = backward(features, hidden_delta, hidden_weight, hidden_bias, LEARNING_RATE)

print(f"hidden weight: {hidden_weight}")
print(f"hidden bias: {hidden_bias}")
print(f"output weight: {output_weight}")
print(f"output bias: {output_bias}")

hidden weight: [[ 1.45353425 -0.14661746]
 [ 1.38767435 -0.33933946]
 [ 2.0254864  -0.21125674]
 [ 0.69613316 -0.14191958]]
hidden bias: [1.00869983 0.0221608  0.79667756 0.75576452]
output weight: [[1.35796276 1.30704936 1.91057461 0.67564236]]
output bias: [1.00777367]


## 验证

### 推理

In [96]:
hidden = forward(test_features, hidden_weight, hidden_bias)
predictions = forward(hidden, output_weight, output_bias)

print(f'predictions: {predictions}')

predictions: [[166.58568956]]


### 评估

In [97]:
error = mse_loss(predictions, test_labels)

print(f'loss: {error}')

loss: 2.5144113737205855


实践证明，两层网络模型是可行的，但是也没有比单层网络模型表现更好，甚至略有损耗。